In [37]:
#Doc https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/introduction_to_SH_APIs.html
#API KEY : https://shapps.dataspace.copernicus.eu/dashboard/#/

In [38]:
#!pip install sentinelhub

In [39]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

#from utils import plot_image

In [40]:
client_id= 'sh-4eaa164d-79a2-48ec-86c8-d0a376a229ee'
client_secret= '4HAs1tuR40KXNHvovCrKoacuab3iTYrm'

In [41]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = client_id
config.sh_client_secret = client_secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [42]:

# Definir el área de interés (AOI) con 4 puntos de lat/lon
lon_min = -63.95
lat_min = -31.75
lon_max = -63.75
lat_max = -31.60

aoi_coords_wgs84 = [lon_min, lat_min, lon_max, lat_max]

In [43]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

Image shape at 10 m resolution: (1883, 1677) pixels


In [44]:
catalog = SentinelHubCatalog(config=config)
catalog

In [45]:
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = "2025-07-01", "2025-07-20"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

Total number of results: 10


[{'id': 'S2A_MSIL2A_20250719T141111_N0511_R110_T20HMK_20250719T211521.SAFE',
  'properties': {'datetime': '2025-07-19T14:21:58.521Z'}},
 {'id': 'S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE',
  'properties': {'datetime': '2025-07-19T14:21:44.092Z'}},
 {'id': 'S2C_MSIL2A_20250717T141111_N0511_R110_T20HMK_20250717T173322.SAFE',
  'properties': {'datetime': '2025-07-17T14:22:02.422Z'}},
 {'id': 'S2C_MSIL2A_20250717T141111_N0511_R110_T20JML_20250717T173322.SAFE',
  'properties': {'datetime': '2025-07-17T14:21:47.995Z'}},
 {'id': 'S2B_MSIL2A_20250712T140709_N0511_R110_T20HMK_20250712T173831.SAFE',
  'properties': {'datetime': '2025-07-12T14:21:42.721Z'}},
 {'id': 'S2B_MSIL2A_20250712T140709_N0511_R110_T20JML_20250712T173831.SAFE',
  'properties': {'datetime': '2025-07-12T14:21:28.273Z'}},
 {'id': 'S2C_MSIL2A_20250707T141111_N0511_R110_T20HMK_20250707T191716.SAFE',
  'properties': {'datetime': '2025-07-07T14:22:01.949Z'}},
 {'id': 'S2C_MSIL2A_20250707T141111_N0511_R110_T

In [46]:
# PRIMERO: Verificar y recargar la configuración
print("=== VERIFICANDO CONFIGURACIÓN ===")
print(f"Config client_id: {config.sh_client_id}")
print(f"Config client_secret: {config.sh_client_secret}")
print(f"Config token_url: {config.sh_token_url}")
print(f"Config base_url: {config.sh_base_url}")

# Si alguno está vacío, volvemos a configurar
if not config.sh_client_id or not config.sh_client_secret:
    print("Reconfiguring...")
    config.sh_client_id = client_id
    config.sh_client_secret = client_secret
    config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    config.sh_base_url = "https://sh.dataspace.copernicus.eu"

# O mejor aún, cargar la configuración guardada
try:
    config = SHConfig("cdse")
    print("✓ Loaded saved configuration")
except:
    print("⚠ Could not load saved config, using current one")

print(f"Final config client_id: {config.sh_client_id}")
print(f"Final config base_url: {config.sh_base_url}")

=== VERIFICANDO CONFIGURACIÓN ===
Config client_id: sh-4eaa164d-79a2-48ec-86c8-d0a376a229ee
Config client_secret: 4HAs1tuR40KXNHvovCrKoacuab3iTYrm
Config token_url: https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token
Config base_url: https://sh.dataspace.copernicus.eu
✓ Loaded saved configuration
Final config client_id: sh-4eaa164d-79a2-48ec-86c8-d0a376a229ee
Final config base_url: https://sh.dataspace.copernicus.eu


In [47]:
# DIAGNÓSTICO: Verificar qué productos tienes
print("=== ANÁLISIS DE PRODUCTOS DESCARGADOS ===")
print("Productos encontrados en results:")
for i, result in enumerate(results):
    product_id = result['id']
    datetime_str = result['properties']['datetime']
    date_str = parse_datetime(datetime_str)
    tile_id = extract_tile_id(product_id)
    satellite = product_id[:3]
    
    print(f"{i+1:2d}. {date_str} | {satellite} | {tile_id} | {product_id}")

print(f"\nTu AOI: {aoi_coords_wgs84}")
print(f"Área: {abs(aoi_coords_wgs84[2] - aoi_coords_wgs84[0]):.3f}° x {abs(aoi_coords_wgs84[3] - aoi_coords_wgs84[1]):.3f}°")

=== ANÁLISIS DE PRODUCTOS DESCARGADOS ===
Productos encontrados en results:
 1. 2025-07-19 | S2A | T20HMK | S2A_MSIL2A_20250719T141111_N0511_R110_T20HMK_20250719T211521.SAFE
 2. 2025-07-19 | S2A | T20JML | S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE
 3. 2025-07-17 | S2C | T20HMK | S2C_MSIL2A_20250717T141111_N0511_R110_T20HMK_20250717T173322.SAFE
 4. 2025-07-17 | S2C | T20JML | S2C_MSIL2A_20250717T141111_N0511_R110_T20JML_20250717T173322.SAFE
 5. 2025-07-12 | S2B | T20HMK | S2B_MSIL2A_20250712T140709_N0511_R110_T20HMK_20250712T173831.SAFE
 6. 2025-07-12 | S2B | T20JML | S2B_MSIL2A_20250712T140709_N0511_R110_T20JML_20250712T173831.SAFE
 7. 2025-07-07 | S2C | T20HMK | S2C_MSIL2A_20250707T141111_N0511_R110_T20HMK_20250707T191716.SAFE
 8. 2025-07-07 | S2C | T20JML | S2C_MSIL2A_20250707T141111_N0511_R110_T20JML_20250707T191716.SAFE
 9. 2025-07-02 | S2B | T20HMK | S2B_MSIL2A_20250702T140709_N0511_R110_T20HMK_20250702T175025.SAFE
10. 2025-07-02 | S2B | T20JML | S2B_MSIL2A

In [48]:
import requests
import os
import zipfile
import json
from datetime import datetime
import rasterio
import numpy as np
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import glob
from pathlib import Path
import xml.etree.ElementTree as ET

def get_cdse_access_token():
    """Obtiene token de acceso para Copernicus Dataspace"""
    
    client_id = 'sh-4eaa164d-79a2-48ec-86c8-d0a376a229ee'
    client_secret = '4HAs1tuR40KXNHvovCrKoacuab3iTYrm'
    
    token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    
    response = requests.post(token_url, data=data)
    if response.status_code == 200:
        access_token = response.json()['access_token']
        print("✓ Access token obtained successfully")
        return access_token
    else:
        print(f"✗ Failed to get access token: {response.status_code}")
        print(response.text)
        return None

def search_sentinel2_products_odata(aoi, start_date, end_date, max_cloud_cover=20):
    """Busca productos Sentinel-2 usando OData API"""
    
    # Convertir AOI a polígono WKT
    lon_min, lat_min, lon_max, lat_max = aoi
    wkt_polygon = f"POLYGON(({lon_min} {lat_min},{lon_max} {lat_min},{lon_max} {lat_max},{lon_min} {lat_max},{lon_min} {lat_min}))"
    
    # URL de búsqueda OData
    base_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
    
    # Filtros de búsqueda
    filters = [
        f"Collection/Name eq 'SENTINEL-2'",
        f"contains(Name,'MSIL2A')",  # Solo productos L2A
        f"OData.CSC.Intersects(area=geography'SRID=4326;{wkt_polygon}')",
        f"ContentDate/Start ge {start_date}T00:00:00.000Z",
        f"ContentDate/Start le {end_date}T23:59:59.999Z"
    ]
    
    # Construir URL de consulta
    filter_string = " and ".join(filters)
    url = f"{base_url}?$filter={filter_string}&$orderby=ContentDate/Start desc&$top=50"
    
    print(f"🔍 Searching for Sentinel-2 products...")
    print(f"    Area: {aoi}")
    print(f"    Period: {start_date} to {end_date}")
    print(f"    Max cloud cover: {max_cloud_cover}%")
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        products = data.get('value', [])
        
        # Filtrar por cobertura de nubes si disponible
        filtered_products = []
        for product in products:
            # Extraer información de cobertura de nubes del nombre del producto si es posible
            name = product.get('Name', '')
            
            # Para Sentinel-2, podemos intentar obtener metadatos adicionales
            filtered_products.append(product)
        
        print(f"✓ Found {len(filtered_products)} Sentinel-2 L2A products")
        return filtered_products
    else:
        print(f"✗ Search failed: {response.status_code}")
        print(response.text)
        return []

def download_sentinel2_product(product_info, access_token, download_dir="sentinel2_complete_tiles"):
    """Descarga un producto completo de Sentinel-2"""
    
    product_id = product_info['Id']
    product_name = product_info['Name']
    
    os.makedirs(download_dir, exist_ok=True)
    
    # URL de descarga
    download_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    zip_filename = os.path.join(download_dir, f"{product_name}.zip")
    
    print(f"📥 Downloading: {product_name}")
    print(f"    Size estimate: ~1-2 GB")
    print(f"    Saving to: {zip_filename}")
    
    # Verificar si ya existe
    if os.path.exists(zip_filename):
        print(f"    ✓ File already exists, skipping download")
        return zip_filename
    
    try:
        # Descargar con streaming para archivos grandes
        with requests.get(download_url, headers=headers, stream=True) as response:
            if response.status_code == 200:
                total_size = int(response.headers.get('content-length', 0))
                downloaded_size = 0
                
                with open(zip_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            downloaded_size += len(chunk)
                            
                            # Progress indicator
                            if total_size > 0:
                                progress = (downloaded_size / total_size) * 100
                                if downloaded_size % (10*1024*1024) == 0:  # Every 10MB
                                    print(f"    Progress: {progress:.1f}% ({downloaded_size/1024/1024:.1f} MB)")
                
                print(f"    ✓ Download completed: {downloaded_size/1024/1024:.1f} MB")
                return zip_filename
            else:
                print(f"    ✗ Download failed: {response.status_code}")
                return None
                
    except Exception as e:
        print(f"    ✗ Download error: {e}")
        return None

def extract_and_process_safe(zip_file, output_dir="processed_tiles"):
    """Extrae y procesa archivo .SAFE de Sentinel-2"""
    
    print(f"📦 Extracting and processing: {os.path.basename(zip_file)}")
    
    extract_dir = zip_file.replace('.zip', '')
    os.makedirs(extract_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        # Extraer ZIP
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        
        # Buscar directorio .SAFE
        safe_dirs = glob.glob(os.path.join(extract_dir, "*.SAFE"))
        if not safe_dirs:
            print(f"    ✗ No .SAFE directory found")
            return None
        
        safe_dir = safe_dirs[0]
        safe_name = os.path.basename(safe_dir)
        
        print(f"    Processing .SAFE: {safe_name}")
        
        # Buscar archivos de imagen (bandas 10m)
        img_data_dir = os.path.join(safe_dir, "GRANULE", "*", "IMG_DATA", "R10m")
        band_files = glob.glob(os.path.join(img_data_dir, "*_B0[234]_10m.jp2"))
        
        if not band_files:
            print(f"    ✗ No 10m resolution bands found")
            return None
        
        print(f"    Found {len(band_files)} band files")
        
        # Procesar bandas RGB (B04, B03, B02)
        rgb_bands = {}
        for band_file in band_files:
            if '_B02_10m.jp2' in band_file:
                rgb_bands['B02'] = band_file  # Blue
            elif '_B03_10m.jp2' in band_file:
                rgb_bands['B03'] = band_file  # Green  
            elif '_B04_10m.jp2' in band_file:
                rgb_bands['B04'] = band_file  # Red
        
        if len(rgb_bands) != 3:
            print(f"    ✗ Missing RGB bands, found: {list(rgb_bands.keys())}")
            return None
        
        # Crear imagen RGB compuesta
        output_filename = f"{safe_name.replace('.SAFE', '')}_RGB_10m.tiff"
        output_path = os.path.join(output_dir, output_filename)
        
        success = create_rgb_composite(rgb_bands, output_path, safe_name)
        
        if success:
            print(f"    ✓ RGB composite created: {output_filename}")
            
            # Limpiar archivos temporales (opcional)
            # shutil.rmtree(extract_dir)
            
            return output_path
        else:
            print(f"    ✗ Failed to create RGB composite")
            return None
            
    except Exception as e:
        print(f"    ✗ Processing error: {e}")
        return None

def create_rgb_composite(rgb_bands, output_path, safe_name):
    """Crea composición RGB desde bandas individuales"""
    
    try:
        print(f"        Creating RGB composite...")
        
        # Leer bandas
        bands_data = {}
        transform = None
        crs = None
        
        for band_name, band_file in rgb_bands.items():
            with rasterio.open(band_file) as src:
                bands_data[band_name] = src.read(1)
                if transform is None:
                    transform = src.transform
                    crs = src.crs
                    height, width = src.shape
        
        # Organizar como RGB (R=B04, G=B03, B=B02)
        rgb_array = np.stack([
            bands_data['B04'],  # Red
            bands_data['B03'],  # Green  
            bands_data['B02']   # Blue
        ], axis=0)
        
        # Normalizar valores (Sentinel-2 L2A ya viene escalado)
        # Aplicar mejora de contraste simple
        rgb_normalized = np.clip(rgb_array * 3.5, 0, 10000).astype(np.uint16)
        
        # Guardar composición RGB
        with rasterio.open(
            output_path, 'w',
            driver='GTiff',
            height=height,
            width=width,
            count=3,
            dtype='uint16',
            crs=crs,
            transform=transform,
            compress='lzw',
            tiled=True,
            blockxsize=512,
            blockysize=512
        ) as dst:
            
            dst.write(rgb_normalized)
            
            # Metadatos
            dst.update_tags(
                PRODUCT_NAME=safe_name,
                BANDS='RGB (B04,B03,B02)',
                RESOLUTION='10m',
                PRODUCT_TYPE='SENTINEL2_L2A_COMPLETE_TILE_RGB'
            )
        
        file_size = os.path.getsize(output_path) / (1024*1024)
        print(f"        ✓ RGB composite saved ({file_size:.1f} MB)")
        
        return True
        
    except Exception as e:
        print(f"        ✗ RGB creation error: {e}")
        return False

def download_complete_sentinel2_tiles_odata(aoi, start_date="2025-07-01", end_date="2025-07-31"):
    """Función principal para descargar tiles completos usando OData API"""
    
    print(f"\n{'='*70}")
    print(f"DOWNLOADING COMPLETE SENTINEL-2 TILES USING ODATA API")
    print(f"{'='*70}")
    print(f"Area of Interest: {aoi}")
    print(f"Date range: {start_date} to {end_date}")
    print(f"{'='*70}")
    
    # 1. Obtener token de acceso
    access_token = get_cdse_access_token()
    if not access_token:
        return
    
    # 2. Buscar productos
    products = search_sentinel2_products_odata(aoi, start_date, end_date)
    if not products:
        print("No products found")
        return
    
    # 3. Descargar y procesar cada producto
    successful_downloads = 0
    failed_downloads = 0
    
    for i, product in enumerate(products[:5], 1):  # Limitar a 5 productos para empezar
        print(f"\n[{i}/5] Processing product: {product['Name']}")
        
        # Descargar producto
        zip_file = download_sentinel2_product(product, access_token)
        
        if zip_file:
            # Procesar archivo .SAFE
            processed_file = extract_and_process_safe(zip_file)
            
            if processed_file:
                successful_downloads += 1
                print(f"✓ Successfully processed: {os.path.basename(processed_file)}")
            else:
                failed_downloads += 1
        else:
            failed_downloads += 1
    
    # Estadísticas finales
    print(f"\n{'='*70}")
    print(f"DOWNLOAD COMPLETED - ODATA API")
    print(f"{'='*70}")
    print(f"✓ Successfully downloaded: {successful_downloads}")
    print(f"✗ Failed downloads: {failed_downloads}")
    print(f"📁 Complete tiles saved in: ./processed_tiles/")
    print(f"📁 Raw .SAFE files in: ./sentinel2_complete_tiles/")
    
    if successful_downloads > 0:
        print(f"\n🎉 Success! Complete Sentinel-2 tiles downloaded and processed!")
        
        # Listar archivos procesados
        if os.path.exists("processed_tiles"):
            files = [f for f in os.listdir("processed_tiles") if f.endswith('.tiff')]
            print(f"\n📁 Processed RGB files:")
            for file in sorted(files):
                file_path = os.path.join("processed_tiles", file)
                size_mb = os.path.getsize(file_path) / (1024*1024)
                print(f"   📄 {file} ({size_mb:.1f} MB)")

# EJECUTAR DESCARGA CON ODATA API
print("🚀 STARTING COMPLETE TILE DOWNLOAD USING ODATA API...")

# Usar tu AOI original
aoi_coords = [-63.95, -31.75, -63.75, -31.6]  # Pergamino area

download_complete_sentinel2_tiles_odata(
    aoi=aoi_coords,
    start_date="2025-07-01", 
    end_date="2025-07-20"
)

🚀 STARTING COMPLETE TILE DOWNLOAD USING ODATA API...

DOWNLOADING COMPLETE SENTINEL-2 TILES USING ODATA API
Area of Interest: [-63.95, -31.75, -63.75, -31.6]
Date range: 2025-07-01 to 2025-07-20
✓ Access token obtained successfully
🔍 Searching for Sentinel-2 products...
    Area: [-63.95, -31.75, -63.75, -31.6]
    Period: 2025-07-01 to 2025-07-20
    Max cloud cover: 20%
✓ Access token obtained successfully
🔍 Searching for Sentinel-2 products...
    Area: [-63.95, -31.75, -63.75, -31.6]
    Period: 2025-07-01 to 2025-07-20
    Max cloud cover: 20%
✓ Found 10 Sentinel-2 L2A products

[1/5] Processing product: S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE
📥 Downloading: S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE
    Size estimate: ~1-2 GB
    Saving to: sentinel2_complete_tiles\S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE.zip
✓ Found 10 Sentinel-2 L2A products

[1/5] Processing product: S2A_MSIL2A_20250719T141111_N0511_R110

KeyboardInterrupt: 